# Anomaly Detection Homework

This notebook is for anomaly detection homework of Applied AI Week 4. The dataset is given with [this link](https://drive.google.com/file/d/1cZGOZu_zdKLXnH-Ap1w9SMffYXZqa2Ot/view?usp=sharing). If you are having problems with the link, contact with me: safak@inzva.com

## Dataset Description
"KDD CUP 99 data set is used mainly to analyze the different
attacks. It consists of nearly 4,900,000 samples with 41
features and each sample is classified as either normal or
attack" [explanation from this source](https://www.ripublication.com/ijaer18/ijaerv13n7_81.pdf)

## Task Description

The dataset is prepared and preprocessed for anomaly detection task, the dataset contains "Probe" and "Normal" targets. "Probe" is anomaly, "Normal" is normal. 

**You are supposed to build a anomaly detection model** with **Vanilla Autoencoder**, **Variational Autoencoder** and **Denoising Autoencoder**. However you are not restricted by autoencoer, you can implement a fancy state-of-the-art ensemble 1000B parameter model. It is really up to you. 

We don't really want you to do sloppy homework.

The variable descriptions:

- train set: kdd_train_probe
- validation set (for hyperparam tuning): kdd_valid_probe
- test set: kdd_test_v2_probe

## What will you report?
Report your average macro f1 score on test set:

```python
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average = "macro")
print(f1)
```


# Preparation (do not edit this part)

In [59]:
import pandas as pd
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning

import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
import sys
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
from tqdm.auto import tqdm

import seaborn as sns
from pylab import rcParams

from sklearn.metrics import f1_score, accuracy_score, classification_report, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from timeit import default_timer as timer

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style="whitegrid", palette="muted", font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 10, 4

In [3]:

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

kdd = pd.read_csv('kdd.csv')
kdd = kdd.iloc[:,1:43]
kdd = kdd.drop(['Protocol Type', 'Service', 'Flag'], axis = 1)

kdd_train = kdd.iloc[0:102563, :]
kdd_test = kdd.iloc[102563:183737, :]

kdd_train_probe = kdd_train[(kdd_train.Type_Groups == 'Normal') | (kdd_train.Type_Groups == 'Probe')]
kdd_test_probe = kdd_test[(kdd_test.Type_Groups == 'Normal') | (kdd_test.Type_Groups == 'Probe')]

kdd_train_probe['Type_Groups'] = np.where(kdd_train_probe['Type_Groups'] == 'Normal', 0, 1)
kdd_test_probe['Type_Groups'] = np.where(kdd_test_probe['Type_Groups'] == 'Normal', 0, 1)

kdd_valid_probe = kdd_test_probe.iloc[14000:34000,:]
kdd_test_v2_probe = pd.concat([kdd_test_probe.iloc[0:14000,:], kdd_test_probe.iloc[34001:64759,:]])


# classify anomalies and normals
# train set: kdd_train_probe
# validation set (for hyperparam tuning): kdd_valid_probe
# test set: kdd_test_v2_probe
# avg. macro f1 score on test set

## Pytorch DataLoaders

In [4]:
# NORMAL: class label 0
# ANOMALY: class label 1

class TabularDataset(Dataset):
    def __init__(self, df):
        super(TabularDataset, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        data = self.df.iloc[idx, :-1].to_numpy()
        return {
            "samples": torch.Tensor(data)
        }
    
class TabularDatasetTest(Dataset):
    def __init__(self, df):
        super(TabularDatasetTest, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        data = self.df.iloc[idx, :-1].to_numpy()
        label = self.df.iloc[idx, -1]
        return {
            "samples": torch.Tensor(data),
            "labels": torch.tensor(label)
        }


BATCH_SIZE = 128

train_normal = kdd_train_probe[kdd_train_probe.Type_Groups == 0]
val_normal = kdd_valid_probe[kdd_valid_probe.Type_Groups == 0]
test_normal = kdd_test_v2_probe[kdd_test_v2_probe.Type_Groups == 0]


train_data = TabularDataset(train_normal)
val_data = TabularDataset(val_normal)
test_data_all = TabularDatasetTest(kdd_test_v2_probe)

# train_dataloader: For training autoencoder. Contains only normal samples
# val_dataloader: For evaluating autoencoder at training phase.
#                 then use it for tune the threshold value.
#                 N.B: setting batch size of 1 at threshold finding phase should be more reasonable:
#                 DataLoader(val_data, shuffle = False, batch_size = 1)
#
# test_all_dataloader: Contains all test samples (anomalies and normals). Use it for
#                      calculating your metrics

# N.B.: finding a threshold value is challenging. iterating all val_dataloader and calculating
#       metrics over it works but it is expensive computationally.

train_dataloader = DataLoader(train_data, shuffle = True, batch_size = BATCH_SIZE)
val_dataloader = DataLoader(val_data, shuffle = False, batch_size = BATCH_SIZE)
test_all_dataloader = DataLoader(test_data_all, shuffle = False, batch_size = 1)

# VAE

In [5]:
# VAE implementation in PyTorch

class LinearVAE(nn.Module):
    def __init__(self, n_features, latent_dim):
        super(LinearVAE, self).__init__()
        self.n_features = n_features

        self.encoder = nn.Sequential(
            nn.Linear(n_features, 20),
            nn.Tanh()
        )

        self.encoder2mean = nn.Linear(20, latent_dim)
        self.encoder2logvar = nn.Linear(20, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 20),
            nn.ReLU(),
            nn.Linear(20, n_features)
        )
    
    def forward(self, x):
        bs = x.size(0)
        out = self.encoder(x)
        mu = self.encoder2mean(out)
        log_var = self.encoder2logvar(out)
        z = self.reparameterize(mu, log_var)
        out = self.decoder(z)
        return out, mu, log_var
        
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        sample = mu + (eps * std)
        return sample

In [6]:
# Checking to see if GPU is avvailable for us
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
# Calculating number of features
num_features = next(iter(train_dataloader))["samples"].shape[-1]
print("number of features: ", num_features)

number of features:  38


In [90]:
# Instantiating VAE
vae = LinearVAE(num_features, latent_dim=5)
vae.to(device)

LinearVAE(
  (encoder): Sequential(
    (0): Linear(in_features=38, out_features=20, bias=True)
    (1): Tanh()
  )
  (encoder2mean): Linear(in_features=20, out_features=5, bias=True)
  (encoder2logvar): Linear(in_features=20, out_features=5, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=38, bias=True)
  )
)

In [88]:
def vaeloss(out, mu, logvar, target):
    # reconstruction loss
    mse_loss = F.mse_loss(out, target, reduction="mean")
    # KL Divergence loss
    # kl = - 0.5 * torch.sum(1 + logvar - torch.square(mu) - torch.square(torch.exp(logvar))) # Most probably incorrect
    kl = - 0.5 * torch.sum(1 + logvar - torch.square(mu) - torch.exp(logvar))
    # Total loss
    loss = mse_loss + kl
    return loss

In [89]:

def evaluate(model, validation_dataloader, criterion=None):
    # implement evaluating function over ```val_dataloader``` variable, to use in training function
    num_validation_batches = len(validation_dataloader)
    loss = 0
    model.eval()
    with torch.no_grad():
        for dictionary in validation_dataloader:
            data = dictionary["samples"].to(device)
            out, mu, sigma = model(data)
            if criterion:
                loss += criterion(out, data)
            else:
                loss += vaeloss(out, mu, sigma, data)
    return loss/num_validation_batches
            


In [91]:

def train(model, training_dataloader, validation_dataloader, validation_loss_tolerance, num_epochs=5):
    # implement training function over ```train_dataloader``` variable
    # do not forget KL divergence :)
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_training_batches = len(training_dataloader)
    num_validation_batches = len(validation_dataloader)
    progress_bar_size = 20.0

    ch = "█"
    intvl = num_training_batches/progress_bar_size;
    valtol = validation_loss_tolerance
    minvalerr = 1000000000.0
    badvalcount = 0

    tStart = timer()
    for epoch in range(num_epochs):
        
        tEpochStart = timer()
        epoch_loss_training = 0.0
        epoch_loss_validation = 0.0
        newnum = 0
        oldnum = 0

        print("Epoch %3d/%3d ["%(epoch+1, num_epochs), end="")
        model.train()
        for i, dictionary in enumerate(training_dataloader):
            data = dictionary["samples"].to(device)
            optimizer.zero_grad()
            predictions, mu, logvar = model(data)
            loss = vaeloss(predictions, mu, logvar, data)
            loss.backward()
            optimizer.step()
            epoch_loss_training += loss.item()
            # Visualization of progressbar
            newnum = int(i/intvl)
            if newnum > oldnum:
                print((newnum-oldnum)*ch, end="")
                oldnum = newnum
        print("] ", end="")
        epoch_loss_training /= num_training_batches

        # epoch_loss_validation = evaluate(model, validation_dataloader, criterion=criterion)
        # epoch_loss_validation = evaluate(model, validation_dataloader, criterion=None)
        model.eval()
        with torch.no_grad():
            for dictionary in validation_dataloader:
                data = dictionary["samples"].to(device)
                predictions, mu, logvar = model(data)
                loss = vaeloss(predictions, mu, logvar, data)
                epoch_loss_validation += loss.item()
        epoch_loss_validation /= num_validation_batches



        tEpochEnd = timer()
        print("Trn Loss: %5.3f |Val Loss: %5.3f |Time: %6.3f sec" % (
            epoch_loss_training, 
            epoch_loss_validation, tEpochEnd-tEpochStart))
        
        # Checking for early stopping
        if epoch_loss_validation < minvalerr:
            minvalerr = epoch_loss_validation
            badvalcount = 0
        else:
            badvalcount += 1
            if badvalcount > valtol:
                print("Validation loss not improved for more than %d epochs."%badvalcount)
                print("Early stopping criterion with validation loss has been reached. Stopping training at %d epochs..."%epoch)
                break

    tFinish = timer()        
    print('Finished Training.')
    print("Training process took %.2f seconds."%(tFinish-tStart))
    print("Saving model...")
    try:
        torch.save(model, "vae_model.pt")
    except Exception as e:
        print(e)
        print("Failed to save the model.")
    print("Done.")
    return model

In [92]:
# Implementation
vae = train(vae, train_dataloader, val_dataloader, 10, 200)

Epoch   1/200 [███████████████████] Trn Loss: 1.269 |Val Loss: 0.685 |Time: 26.219 sec
Epoch   2/200 [███████████████████] Trn Loss: 0.586 |Val Loss: 0.654 |Time: 26.093 sec
Epoch   3/200 [███████████████████] Trn Loss: 0.576 |Val Loss: 0.646 |Time: 26.198 sec
Epoch   4/200 [███████████████████] Trn Loss: 0.572 |Val Loss: 0.639 |Time: 26.164 sec
Epoch   5/200 [███████████████████] Trn Loss: 0.571 |Val Loss: 0.635 |Time: 26.082 sec
Epoch   6/200 [███████████████████] Trn Loss: 0.569 |Val Loss: 0.633 |Time: 26.140 sec
Epoch   7/200 [███████████████████] Trn Loss: 0.569 |Val Loss: 0.632 |Time: 26.110 sec
Epoch   8/200 [███████████████████] Trn Loss: 0.569 |Val Loss: 0.631 |Time: 26.100 sec
Epoch   9/200 [███████████████████] Trn Loss: 0.568 |Val Loss: 0.629 |Time: 26.014 sec
Epoch  10/200 [███████████████████] Trn Loss: 0.568 |Val Loss: 0.628 |Time: 26.175 sec
Epoch  11/200 [███████████████████] Trn Loss: 0.568 |Val Loss: 0.627 |Time: 26.045 sec
Epoch  12/200 [███████████████████] Trn Los

In [93]:

def calculate_f1_score(model, testing_dataloader, threshold):
    # implement metric function to calculate macro f1 score over ```test_all_dataloader```
    # by using predefined threshold value
    # if overall loss > threshold, then it is anomaly; else normal
    tot_loss = 0
    model.cpu()
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for dictionary in testing_dataloader:
            data = dictionary["samples"]
            label = dictionary["labels"]
            out, _, _ = model(data)
            # reconstruction loss
            loss = mean_squared_error(data, out)
            pred = 1 if loss > threshold else 0
            preds.append(pred)
            trues.append(label[0])
    preds = np.array(preds)
    trues = np.array(trues)
    f1 = f1_score(trues,preds)
    return f1

In [94]:
vae_f1 = calculate_f1_score(vae, test_all_dataloader, 1)
print(vae_f1)

0.7886486486486486


# Vanilla AE

In [80]:
class VanillaAE(nn.Module):
    # implement vanilla autoencoder in PyTorch
    def __init__(self, sizevec):
        super(VanillaAE, self).__init__()
        assert(sizevec[0]==sizevec[-1])
        self.layers = []
        self.sizevec = sizevec
        old = sizevec[0]
        new = sizevec[1]
        for i,size in enumerate(sizevec[1:]):
            self.layers.append(nn.Linear(old, new))
            self.layers.append(nn.ReLU())
            old = new
            if i < len(sizevec)-2:
                new = sizevec[i+2]
        self.net = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.net(x)
    

In [81]:
# Instantiation
sizevec = [num_features, 10, 5, 10, num_features]
vanae = VanillaAE(sizevec)
vanae.to(device)

VanillaAE(
  (net): Sequential(
    (0): Linear(in_features=38, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=38, bias=True)
    (7): ReLU()
  )
)

In [82]:
def evaluate(model, dataloader, criterion):
    # implement evaluating function over ```val_dataloader``` variable, to use in training function
    model.eval()
    tot_loss = 0
    num_batches = len(dataloader)
    with torch.no_grad():
        for dictionary in dataloader:
            data = dictionary["samples"].to(device)
            out = model(data)
            tot_loss += criterion(data, out)
    tot_loss /= num_batches
    return tot_loss

In [83]:
def train(model, training_dataloader, validation_dataloader, validation_loss_tolerance, num_epochs):
    # implement training function over ```train_dataloader``` variable
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    num_training_batches = len(training_dataloader)
    num_validation_batches = len(validation_dataloader)
    progress_bar_size = 20.0

    ch = "█"
    intvl = num_training_batches/progress_bar_size;
    valtol = validation_loss_tolerance
    minvalerr = 1000000000.0
    badvalcount = 0

    tStart = timer()
    for epoch in range(num_epochs):
        
        tEpochStart = timer()
        epoch_loss_training = 0.0
        epoch_loss_validation = 0.0
        newnum = 0
        oldnum = 0

        print("Epoch %3d/%3d ["%(epoch+1, num_epochs), end="")
        model.train()
        for i, dictionary in enumerate(training_dataloader):
            data = dictionary["samples"].to(device)
            optimizer.zero_grad()
            predictions = model(data)
            loss = criterion(data, predictions)
            loss.backward()
            optimizer.step()
            epoch_loss_training += loss.item()
            # Visualization of progressbar
            newnum = int(i/intvl)
            if newnum > oldnum:
                print((newnum-oldnum)*ch, end="")
                oldnum = newnum
        print("] ", end="")
        epoch_loss_training /= num_training_batches

        epoch_loss_validation = evaluate(model, validation_dataloader, criterion)

        tEpochEnd = timer()
        print("Trn Loss: %5.3f |Val Loss: %5.3f |Time: %6.3f sec" % (
            epoch_loss_training, 
            epoch_loss_validation, tEpochEnd-tEpochStart))
        
        # Checking for early stopping
        if epoch_loss_validation < minvalerr:
            minvalerr = epoch_loss_validation
            badvalcount = 0
        else:
            badvalcount += 1
            if badvalcount > valtol:
                print("Validation loss not improved for more than %d epochs."%badvalcount)
                print("Early stopping criterion with validation loss has been reached. Stopping training at %d epochs..."%epoch)
                break

    tFinish = timer()        
    print('Finished Training.')
    print("Training process took %.2f seconds."%(tFinish-tStart))
    print("Saving model...")
    try:
        torch.save(model, "vanae_model.pt")
    except Exception as e:
        print(e)
        print("Failed to save the model.")
    print("Done.")
    return model

In [84]:
# Implementation
vanae = train(vanae, train_dataloader, val_dataloader, 10, 200)

Epoch   1/200 [███████████████████] Trn Loss: 0.488 |Val Loss: 0.505 |Time: 25.055 sec
Epoch   2/200 [███████████████████] Trn Loss: 0.397 |Val Loss: 0.468 |Time: 25.049 sec
Epoch   3/200 [███████████████████] Trn Loss: 0.362 |Val Loss: 0.410 |Time: 25.538 sec
Epoch   4/200 [███████████████████] Trn Loss: 0.338 |Val Loss: 0.358 |Time: 24.951 sec
Epoch   5/200 [███████████████████] Trn Loss: 0.313 |Val Loss: 0.325 |Time: 25.019 sec
Epoch   6/200 [███████████████████] Trn Loss: 0.298 |Val Loss: 0.270 |Time: 24.982 sec
Epoch   7/200 [███████████████████] Trn Loss: 0.285 |Val Loss: 0.246 |Time: 24.973 sec
Epoch   8/200 [███████████████████] Trn Loss: 0.279 |Val Loss: 0.242 |Time: 25.032 sec
Epoch   9/200 [███████████████████] Trn Loss: 0.278 |Val Loss: 0.237 |Time: 25.140 sec
Epoch  10/200 [███████████████████] Trn Loss: 0.276 |Val Loss: 0.244 |Time: 25.134 sec
Epoch  11/200 [███████████████████] Trn Loss: 0.274 |Val Loss: 0.241 |Time: 25.076 sec
Epoch  12/200 [███████████████████] Trn Los

In [85]:
def calculate_f1_score(model, testing_dataloader, threshold):
    # implement metric function to calculate macro f1 score over ```test_all_dataloader```
    # by using predefined threshold value
    # if overall loss > threshold, then it is anomaly; else normal
    tot_loss = 0
    model.cpu()
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for dictionary in testing_dataloader:
            data = dictionary["samples"]
            label = dictionary["labels"]
            out = model(data)
            # reconstruction loss
            loss = mean_squared_error(data, out)
            pred = 1 if loss > threshold else 0
            preds.append(pred)
            trues.append(label[0])
    preds = np.array(preds)
    trues = np.array(trues)
    f1 = f1_score(trues,preds)
    return f1

In [86]:
vanae_f1 = calculate_f1_score(vanae, test_all_dataloader, 1.5)
print(vanae_f1)

0.8965271593944791


# DAE

In [56]:
class DenoisingAE(nn.Module):
    # implement denoising autoencoder in PyTorch
    def __init__(self, sizevec):
        super(DenoisingAE, self).__init__()
        assert(sizevec[0]==sizevec[-1])
        self.layers = []
        self.sizevec = sizevec
        old = sizevec[0]
        new = sizevec[1]
        for i,size in enumerate(sizevec[1:]):
            self.layers.append(nn.Linear(old, new))
            self.layers.append(nn.ReLU())
            old = new
            if i < len(sizevec)-2:
                new = sizevec[i+2]
        self.net = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.net(x)

In [57]:
# Instantiation
sizevec = [num_features, 10, 5, 10, num_features]
dae = DenoisingAE(sizevec)
dae.to(device)

DenoisingAE(
  (net): Sequential(
    (0): Linear(in_features=38, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=38, bias=True)
    (7): ReLU()
  )
)

In [64]:
def evaluate(model, dataloader, criterion, noisiness):
    # implement evaluating function over ```val_dataloader``` variable, to use in training function
    model.eval()
    tot_loss = 0
    num_batches = len(dataloader)
    with torch.no_grad():
        for dictionary in dataloader:
            data = dictionary["samples"].to(device)
            data += torch.randn_like(data)*noisiness
            out = model(data)
            tot_loss += criterion(data, out)
    tot_loss /= num_batches
    return tot_loss

In [65]:
def train(model, training_dataloader, validation_dataloader, validation_loss_tolerance, num_epochs, noisiness):
    # implement training function over ```train_dataloader``` variable
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    num_training_batches = len(training_dataloader)
    num_validation_batches = len(validation_dataloader)
    progress_bar_size = 20.0

    ch = "█"
    intvl = num_training_batches/progress_bar_size;
    valtol = validation_loss_tolerance
    minvalerr = 1000000000.0
    badvalcount = 0

    tStart = timer()
    for epoch in range(num_epochs):
        
        tEpochStart = timer()
        epoch_loss_training = 0.0
        epoch_loss_validation = 0.0
        newnum = 0
        oldnum = 0

        print("Epoch %3d/%3d ["%(epoch+1, num_epochs), end="")
        model.train()
        for i, dictionary in enumerate(training_dataloader):
            data = dictionary["samples"].to(device)
            data += torch.randn_like(data)*noisiness
            optimizer.zero_grad()
            predictions = model(data)
            loss = criterion(data, predictions)
            loss.backward()
            optimizer.step()
            epoch_loss_training += loss.item()
            # Visualization of progressbar
            newnum = int(i/intvl)
            if newnum > oldnum:
                print((newnum-oldnum)*ch, end="")
                oldnum = newnum
        print("] ", end="")
        epoch_loss_training /= num_training_batches

        epoch_loss_validation = evaluate(model, validation_dataloader, criterion, noisiness)

        tEpochEnd = timer()
        print("Trn Loss: %5.3f |Val Loss: %5.3f |Time: %6.3f sec" % (
            epoch_loss_training, 
            epoch_loss_validation, tEpochEnd-tEpochStart))
        
        # Checking for early stopping
        if epoch_loss_validation < minvalerr:
            minvalerr = epoch_loss_validation
            badvalcount = 0
        else:
            badvalcount += 1
            if badvalcount > valtol:
                print("Validation loss not improved for more than %d epochs."%badvalcount)
                print("Early stopping criterion with validation loss has been reached. Stopping training at %d epochs..."%epoch)
                break

    tFinish = timer()        
    print('Finished Training.')
    print("Training process took %.2f seconds."%(tFinish-tStart))
    print("Saving model...")
    try:
        torch.save(model, "dae_model.pt")
    except Exception as e:
        print(e)
        print("Failed to save the model.")
    print("Done.")
    return model

In [66]:
# Implementation
dae = train(dae, train_dataloader, val_dataloader, 10, 200, 0.1)

Epoch   1/200 [███████████████████] Trn Loss: 0.412 |Val Loss: 0.563 |Time: 25.907 sec
Epoch   2/200 [███████████████████] Trn Loss: 0.393 |Val Loss: 0.539 |Time: 26.119 sec
Epoch   3/200 [███████████████████] Trn Loss: 0.374 |Val Loss: 0.510 |Time: 25.306 sec
Epoch   4/200 [███████████████████] Trn Loss: 0.358 |Val Loss: 0.500 |Time: 25.320 sec
Epoch   5/200 [███████████████████] Trn Loss: 0.355 |Val Loss: 0.496 |Time: 25.225 sec
Epoch   6/200 [███████████████████] Trn Loss: 0.352 |Val Loss: 0.494 |Time: 25.099 sec
Epoch   7/200 [███████████████████] Trn Loss: 0.351 |Val Loss: 0.492 |Time: 25.041 sec
Epoch   8/200 [███████████████████] Trn Loss: 0.352 |Val Loss: 0.491 |Time: 24.965 sec
Epoch   9/200 [███████████████████] Trn Loss: 0.350 |Val Loss: 0.493 |Time: 25.033 sec
Epoch  10/200 [███████████████████] Trn Loss: 0.348 |Val Loss: 0.492 |Time: 25.092 sec
Epoch  11/200 [███████████████████] Trn Loss: 0.343 |Val Loss: 0.489 |Time: 25.051 sec
Epoch  12/200 [███████████████████] Trn Los

In [67]:
def calculate_f1_score(model, testing_dataloader, threshold):
    # implement metric function to calculate macro f1 score over ```test_all_dataloader```
    # by using predefined threshold value
    # if overall loss > threshold, then it is anomaly; else normal
    tot_loss = 0
    model.cpu()
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for dictionary in testing_dataloader:
            data = dictionary["samples"]
            label = dictionary["labels"]
            out = model(data)
            # reconstruction loss
            loss = mean_squared_error(data, out)
            pred = 1 if loss > threshold else 0
            preds.append(pred)
            trues.append(label[0])
    preds = np.array(preds)
    trues = np.array(trues)
    f1 = f1_score(trues,preds)
    return f1

In [79]:
dae_f1 = calculate_f1_score(dae, test_all_dataloader, 1.5)
print(dae_f1)

0.8188768009408998
